In [1]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import tomllib
# from bart import (
#     process_BART_county,
#     process_BART_data,
#     process_BART_Screenline,
#     read_nodes,
#     station_name,
# )
# from map_data import process_bart_map, process_muni_map
from muni import process_muni, read_transit_lines
from obs import process_obs_data
# from screen import concat_final_Screenline
# from simwrapper_table import (
#     process_data,
#     process_mkd_bart,
#     process_mkd_muni,
#     process_mkd_screenline,
# )
# from total_val import (
#     assign_ferry_name,
#     assign_muni_name,
#     process_valTotal_Operator,
#     process_valTotal_Submode,
# )
from transit_function import read_dbf_and_groupby_sum, read_transit_assignments

# from transit import transit_validation_2019_alfaro_filepath


In [98]:
with open("transit.toml", "rb") as f:
    config = tomllib.load(f)

model_run_dir = Path(config["directories"]["model_run"])
markdown_output_dir = Path(config["directories"]["markdown_output_dir"])
transit_input_dir = Path(config["directories"]["transit_input_dir"])
observed_BART = Path(config["transit"]["observed_BART"])
observed_BART_county = Path(config["transit"]["observed_BART_county"])
observed_BART_Screenline = Path(config["transit"]["observed_BART_Screenline"])
observed_MUNI_Line = Path(config["transit"]["observed_MUNI_Line"])
observed_Screenline = Path(config["transit"]["observed_Screenline"])
observed_NTD = Path(config["transit"]["observed_NTD"])
model_BART = Path(config["output"]["model_BART"])
model_BART_county = Path(config["output"]["model_BART_county"])
model_BART_Screenline = Path(config["output"]["model_BART_Screenline"])
model_MUNI_Line = Path(config["output"]["model_MUNI_Line"])
model_Screenline = Path(config["output"]["model_Screenline"])
output_dir = model_run_dir / "validation_workbook" / "output"
output_transit_dir = output_dir / "transit"
transit_validation_2019_alfaro_filepath = config["transit"][
    "transit_validation_2019_alfaro_filepath"
]
transit_line_rename_filepath = (
    Path(config["directories"]["resources"]) / config["transit"]["line_rename_filename"]
)
time_periods = ["EA", "AM", "MD", "PM", "EV"]
tod_order = ["EA", "AM", "MD", "PM", "EV", "Total"]


In [3]:
combined_gdf = read_transit_assignments(model_run_dir, time_periods)

Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAEA.DBF
Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAAM.DBF
Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAMD.DBF
Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAPM.DBF
Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAEV.DBF


In [131]:
from shapely.geometry import LineString, Point

In [147]:
df_station_name = pd.read_csv(transit_input_dir / "station_node_match.csv")
df_station_name["geometry"] = df_station_name.apply(lambda row: Point(row['x'], row['y']), axis=1)
station = gpd.GeoDataFrame(df_station_name, geometry="geometry")
station


,Station,Node,County,x,y,geometry
0,12TH,16509,Alameda,6049767.140,2119506.717,POINT (6049767.140 2119506.717)
1,16TH,16515,San Francisco,6006767.683,2106509.658,POINT (6006767.683 2106509.658)
2,19TH,16508,Alameda,6050867.274,2121607.926,POINT (6050867.274 2121607.926)
3,24TH,16516,San Francisco,6007065.529,2101807.039,POINT (6007065.529 2101807.039)
4,ANTC,15231,Contra Costa,6013668.588,2116709.291,POINT (6013668.588 2116709.291)
5,ASHB,16525,Alameda,6012268.724,2115308.761,POINT (6012268.724 2115308.761)
6,BALB,16518,San Francisco,6010767.723,2113809.161,POINT (6010767.723 2113809.161)
7,BAYF,16530,Alameda,6008766.210,2112007.768,POINT (6008766.210 2112007.768)
8,CAST,16537,Alameda,6002267.433,2095109.920,POINT (6002267.433 2095109.920)
9,CIVC,16514,San Francisco,5998566.868,2090908.075,POINT (5998566.868 2090908.075)


In [154]:
# Convert station abbreviation to full name
abbr_to_full = {
    "12TH": "Oakland City Center",
    "16TH": "16th/Mission",
    "19TH": "19th St Oakland",
    "24TH": "24th/Mission",
    "ANTC": "ANTC",  # NOT IN MODEL (Antioch)
    "ASHB": "Ashby",
    "BALB": "Balboa Park",
    "BAYF": "Bay Fair",
    "BERY": "Berryessa",
    "CAST": "Castro Valley",
    "CIVC": "Civic Center",
    "COLM": "Colma",
    "COLS": "Coliseum OAK",
    "CONC": "Concord",
    "DALY": "Daly City",
    "DBRK": "Downtown Berkeley",
    "DELN": "El Cerrito del Norte",
    "DUBL": "Dublin/Pleasanton",
    "EMBR": "Embarcadero",
    "FRMT": "Fremont",
    "FTVL": "Fruitvale",
    "GLEN": "Glen Park",
    "HAYW": "Hayward",
    "LAFY": "Lafayette",
    "LAKE": "Lake Merritt",
    "MCAR": "MacArthur",
    "MLBR": "Millbrae",
    "MONT": "Montgomery",
    "MLPT": "Milpitas",
    "NBRK": "North Berkeley",
    "NCON": "North Concord",
    "OAKL": "OAKL",  # NOT IN MODEL (Oakland International Airport)
    "ORIN": "Orinda",
    "PCTR": "PCTR",  # NOT IN MODEL (Pittsburg Center Station)
    "PHIL": "Pleasant Hill",
    "PITT": "Pittsburg/Bay Point",
    "PLZA": "El Cerrito Plaza",
    "POWL": "Powell",
    "RICH": "Richmond",
    "ROCK": "Rockridge",
    "SANL": "San Leandro",
    "SBRN": "San Bruno",
    "SFIA": "SFO",
    "SHAY": "S Hayward",
    "SSAN": "South SF",
    "UCTY": "Union City",
    "WARM": "Warm Springs",
    "WCRK": "Walnut Creek",
    "WDUB": "West Dublin",
    "WOAK": "W Oakland",
}

station_name = {
    "Station": [
        "12TH",
        "16TH",
        "19TH",
        "24TH",
        "ANTC",
        "ASHB",
        "BALB",
        "BAYF",
        "CAST",
        "CIVC",
        "COLM",
        "COLS",
        "CONC",
        "DALY",
        "DBRK",
        "DELN",
        "DUBL",
        "EMBR",
        "FRMT",
        "FTVL",
        "GLEN",
        "HAYW",
        "LAFY",
        "LAKE",
        "MCAR",
        "MLBR",
        "MONT",
        "NBRK",
        "NCON",
        "OAKL",
        "ORIN",
        "PCTR",
        "PHIL",
        "PITT",
        "PLZA",
        "POWL",
        "RICH",
        "ROCK",
        "SANL",
        "SBRN",
        "SFIA",
        "SHAY",
        "SSAN",
        "UCTY",
        "WARM",
        "WCRK",
        "WDUB",
        "WOAK",
        "MLPT",
        "BERY",
    ],
    "Node": [
        16509,
        16515,
        16508,
        16516,
        15231,
        16525,
        16518,
        16530,
        16537,
        16514,
        16539,
        16532,
        16501,
        16519,
        16523,
        16521,
        16538,
        16511,
        16526,
        16533,
        16517,
        16529,
        16504,
        16534,
        16507,
        16543,
        16512,
        16524,
        16535,
        16000,
        16505,
        15230,
        16502,
        16536,
        16522,
        16513,
        16520,
        16506,
        16531,
        16541,
        16542,
        16528,
        16540,
        16527,
        16544,
        16503,
        16545,
        16510,
        11093,
        15203,
    ],
}

station_node = {
    "Node": [
        16509,
        16515,
        16508,
        16516,
        15231,
        16525,
        16518,
        16530,
        16537,
        16514,
        16539,
        16532,
        16501,
        16519,
        16523,
        16521,
        16538,
        16511,
        16526,
        16533,
        16517,
        16529,
        16504,
        16534,
        16507,
        16543,
        16512,
        16524,
        16535,
        16000,
        16505,
        15230,
        16502,
        16536,
        16522,
        16513,
        16520,
        16506,
        16531,
        16541,
        16542,
        16528,
        16540,
        16527,
        16544,
        16503,
        16545,
        16510,
        11093,
        15203,
    ],
    # Find geometry in X:\Projects\ConnectSF\2050_Baseline
    "geometry": [
        (6049767.14, 2119506.7165),
        (6006767.6834, 2106509.6581),
        (6050867.2736, 2121607.9255),
        (6007065.5285, 2101807.0389),
        (6013668.588, 2116709.2907),
        (6012268.7237, 2115308.7612),
        (6010767.7227, 2113809.1607),
        (6008766.21, 2112007.7684),
        (6002267.4332, 2095109.9198),
        (5998566.8683, 2090908.0746),
        (5992068.011, 2085310.1909),
        (6190907.33, 2188926.7452),
        (6050568.4709, 2138306.2913),
        (6122066.435, 2040507.4321),
        (6090665.2322, 2081908.2387),
        (6105865.8549, 2078508.3059),
        (5992566.8931, 2077808.348),
        (6071765.9655, 2101207.5399),
        (6120867.9426, 2181006.9044),
        (6051067.9389, 2144707.7734),
        (6037763.8869, 2164902.414),
        (6157366.8859, 2081006.495),
        (6133866.765, 2027908.4692),
        (6063066.6973, 2109207.3256),
        (6102965.7997, 2071205.9733),
        (6092368.2914, 2152007.3679),
        (6051067.7192, 2117807.513),
        (6051867.9655, 2128708.394),
        (6015665.034, 2045809.4521),
        (6046968.3673, 2145106.2995),
        (6122569.3244, 2191706.0269),
        (6061416.3918, 2091921.0042),
        (6075866.7507, 2146907.2611),
        (6161632.1229, 2195698.4677),
        (6112666.5933, 2163804.4841),
        (6146166.9892, 2195906.2658),
        (6042168.6724, 2155907.4969),
        (6027067.3566, 2168806.3649),
        (6055968.0873, 2135207.2884),
        (6081167.6265, 2090007.967),
        (6006665.742, 2060908.5058),
        (6014967.7221, 2051908.139),
        (6111466.3558, 2057608.1106),
        (5999066.9282, 2069808.0747),
        (6142865.7593, 2009107.7673),
        (6109469.0872, 2155705.9545),
        (6148868.323, 2080406.2284),
        (6043866.7193, 2120206.1134),
        (6156866.4816, 1974108.2663),
        (6162684.7021, 1961206.4605),
    ],
}


df_station_name = pd.DataFrame(station_name)
# df_station_name["Station_name"] = df_station_name["Station"].apply(
#     lambda x: abbr_to_full[x]
# )
# station_name["geometry"] = [Point(xy) for xy in station_name["geometry"]]

# # station = gpd.GeoDataFrame(station_name, geometry="geometry")
# # station = station.merge(df_station_name, on="Station", how="left")
# # station
df_station_name


,Node,geometry
0,16509,"(6049767.14, 2119506.7165)"
1,16515,"(6006767.6834, 2106509.6581)"
2,16508,"(6050867.2736, 2121607.9255)"
3,16516,"(6007065.5285, 2101807.0389)"
4,15231,"(6013668.588, 2116709.2907)"
5,16525,"(6012268.7237, 2115308.7612)"
6,16518,"(6010767.7227, 2113809.1607)"
7,16530,"(6008766.21, 2112007.7684)"
8,16537,"(6002267.4332, 2095109.9198)"
9,16514,"(5998566.8683, 2090908.0746)"


In [ ]:
from shapely.geometry import LineString, Point

In [130]:
station_name = {
    "Node": [
        16509,
        16515,
        16508,
        16516,
        15231,
        16525,
        16518,
        16530,
        16537,
        16514,
        16539,
        16532,
        16501,
        16519,
        16523,
        16521,
        16538,
        16511,
        16526,
        16533,
        16517,
        16529,
        16504,
        16534,
        16507,
        16543,
        16512,
        16524,
        16535,
        16000,
        16505,
        15230,
        16502,
        16536,
        16522,
        16513,
        16520,
        16506,
        16531,
        16541,
        16542,
        16528,
        16540,
        16527,
        16544,
        16503,
        16545,
        16510,
        11093,
        15203,
    ],
    # Find geometry in X:\Projects\ConnectSF\2050_Baseline
    "geometry": [
        (6049767.14, 2119506.7165),
        (6006767.6834, 2106509.6581),
        (6050867.2736, 2121607.9255),
        (6007065.5285, 2101807.0389),
        (6013668.588, 2116709.2907),
        (6012268.7237, 2115308.7612),
        (6010767.7227, 2113809.1607),
        (6008766.21, 2112007.7684),
        (6002267.4332, 2095109.9198),
        (5998566.8683, 2090908.0746),
        (5992068.011, 2085310.1909),
        (6190907.33, 2188926.7452),
        (6050568.4709, 2138306.2913),
        (6122066.435, 2040507.4321),
        (6090665.2322, 2081908.2387),
        (6105865.8549, 2078508.3059),
        (5992566.8931, 2077808.348),
        (6071765.9655, 2101207.5399),
        (6120867.9426, 2181006.9044),
        (6051067.9389, 2144707.7734),
        (6037763.8869, 2164902.414),
        (6157366.8859, 2081006.495),
        (6133866.765, 2027908.4692),
        (6063066.6973, 2109207.3256),
        (6102965.7997, 2071205.9733),
        (6092368.2914, 2152007.3679),
        (6051067.7192, 2117807.513),
        (6051867.9655, 2128708.394),
        (6015665.034, 2045809.4521),
        (6046968.3673, 2145106.2995),
        (6122569.3244, 2191706.0269),
        (6061416.3918, 2091921.0042),
        (6075866.7507, 2146907.2611),
        (6161632.1229, 2195698.4677),
        (6112666.5933, 2163804.4841),
        (6146166.9892, 2195906.2658),
        (6042168.6724, 2155907.4969),
        (6027067.3566, 2168806.3649),
        (6055968.0873, 2135207.2884),
        (6081167.6265, 2090007.967),
        (6006665.742, 2060908.5058),
        (6014967.7221, 2051908.139),
        (6111466.3558, 2057608.1106),
        (5999066.9282, 2069808.0747),
        (6142865.7593, 2009107.7673),
        (6109469.0872, 2155705.9545),
        (6148868.323, 2080406.2284),
        (6043866.7193, 2120206.1134),
        (6156866.4816, 1974108.2663),
        (6162684.7021, 1961206.4605),
    ],
}

In [133]:
station_name["geometry"] = [Point(xy) for xy in station_name["geometry"]]

In [134]:
station_name

{'Node': [16509,
  16515,
  16508,
  16516,
  15231,
  16525,
  16518,
  16530,
  16537,
  16514,
  16539,
  16532,
  16501,
  16519,
  16523,
  16521,
  16538,
  16511,
  16526,
  16533,
  16517,
  16529,
  16504,
  16534,
  16507,
  16543,
  16512,
  16524,
  16535,
  16000,
  16505,
  15230,
  16502,
  16536,
  16522,
  16513,
  16520,
  16506,
  16531,
  16541,
  16542,
  16528,
  16540,
  16527,
  16544,
  16503,
  16545,
  16510,
  11093,
  15203],
 'geometry': [<POINT (6.05e+06 2.12e+06)>,
  <POINT (6.01e+06 2.11e+06)>,
  <POINT (6.05e+06 2.12e+06)>,
  <POINT (6.01e+06 2.1e+06)>,
  <POINT (6.01e+06 2.12e+06)>,
  <POINT (6.01e+06 2.12e+06)>,
  <POINT (6.01e+06 2.11e+06)>,
  <POINT (6.01e+06 2.11e+06)>,
  <POINT (6e+06 2.1e+06)>,
  <POINT (6e+06 2.09e+06)>,
  <POINT (5.99e+06 2.09e+06)>,
  <POINT (6.19e+06 2.19e+06)>,
  <POINT (6.05e+06 2.14e+06)>,
  <POINT (6.12e+06 2.04e+06)>,
  <POINT (6.09e+06 2.08e+06)>,
  <POINT (6.11e+06 2.08e+06)>,
  <POINT (5.99e+06 2.08e+06)>,
  <POINT (